In [0]:
from sklearn import preprocessing
import pandas as pd
# import tsfresh
# from tsfresh import extract_features
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from google.colab import drive
import xgboost as xgb
from sklearn.metrics import accuracy_score
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Read Train and Test files

In [0]:
df_train = pd.read_csv('/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/Processed_files/v2/All_Train_v1.0.2.csv')
df_test =  pd.read_csv('/content/drive/My Drive/BEAT_PD/Data/CIS-PD/test_data/Processed_files/v3/All_Test_v1.0.2.csv')
df_train.head()

,Unnamed: 0,measurement_id,subject_id,on_off,dyskinesia,tremor,AVG_x,AVG_y,AVG_z,STD_x,STD_y,STD_z,RMS_x,RMS_y,RMS_z,MinMax_x,MinMax_y,MinMax_z,FiltSTD3_x,FiltSTD3_y,FiltSTD3_z,Corr_yz,Peaks50hz_x,Peaks50hz_y,Peaks50hz_z,skew_x,skew_y,skew_z,kurt_x,kurt_y,kurt_z,AVG_SMA,STD_SMA,RMS_SMA,MinMax_SMA,FiltSTD3_SMA,Peaks50hz_SMA,skew_SMA,kurt_SMA,Min_SMA_BP,Max_SMA_BP,STD_SMA_BP,Med_SMA_BP,skew_SMA_BP,kurt_SMA_BP,FreqMaxPSD_SMA_BP,Peaks50hz_SMA_BP,AVG_MAXPeaks_SMA_BP,STD_MAXPeaks_SMA_BP,VAR_MAXPeaks_SMA_BP,Med_MAXPeaks_SMA_BP,DistMaxPeaksSMA_1,DistMaxPeaksSMA_2,DistMaxPeaksSMA_3,DistMaxPeaksSMA_4,Abs_BP_SMA_1_to_3Hz,Abs_BP_SMA_3_to_7Hz,Abs_BP_SMA_1_to_10Hz,Abs_BP_SMA_3_to_10Hz,Rel_BP_SMA_1_to_3Hz,Rel_BP_SMA_3_to_7Hz,Rel_BP_SMA_1_to_10Hz,Rel_BP_SMA_3_to_10Hz
0,0,cc7b822c-e310-46f0-a8ea-98c95fdb67a1,1004,1.0,1.0,1.0,0.054452,0.322197,0.698873,0.373653,0.407493,0.405940,0.377600,0.519482,0.808214,8.769043,9.460938,6.954834,854.0,854.0,755.0,0.509150,5.0,55.0,6.0,-0.968443,0.470836,1.409108,6.400680,1.716051,2.642136,1.016154,0.154064,1.032305,7.489629,745.0,228.0,5.819569,94.146477,-2.778404,2.761872,0.154064,7.484231e-06,-0.558953,30.268464,1.8,45.0,13814.800000,9836.083497,9.674854e+07,13003.0,514.0,211.0,236.0,236.0,1.660314e-02,8.134745e-03,0.027674,1.107121e-02,0.520460,0.255001,0.867511,0.347050
1,1,5163afe8-a6b0-4ea4-b2ba-9b4501dd5912,1004,0.0,0.0,0.0,0.074134,0.081874,0.823729,0.292569,0.285953,0.372113,0.301815,0.297443,0.903879,10.599854,4.578369,7.179688,757.0,757.0,968.0,0.750197,1.0,12.0,4.0,-1.082600,1.562281,1.998585,9.976239,3.981422,3.415090,0.994074,0.074029,0.998280,7.223960,908.0,301.0,15.646557,600.502892,-1.797153,2.208451,0.074029,6.830130e-07,0.834695,147.333891,1.7,17.0,46845.294118,11748.681166,1.380315e+08,49614.0,47939.0,391.0,181.0,181.0,5.844152e-07,5.962829e-07,0.000001,9.129417e-07,0.344747,0.351748,0.883292,0.538545
2,2,5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a,1004,1.0,1.0,1.0,0.791880,0.273321,0.197197,0.197583,0.394550,0.260380,0.816158,0.479973,0.326626,6.768799,4.093750,6.220703,599.0,599.0,707.0,-0.247554,2.0,8.0,14.0,0.928359,-0.206234,2.621623,33.204894,-1.445806,11.622991,0.997105,0.078579,1.001585,5.255477,616.0,303.0,16.273824,429.727838,-2.043263,2.412209,0.078579,1.441539e-05,3.112186,177.418947,1.8,19.0,29583.684211,10419.452976,1.085650e+08,31769.0,645.0,28246.0,177.0,177.0,2.901085e-03,2.333237e-03,0.006816,3.914435e-03,0.399039,0.320932,0.937462,0.538423
3,3,fb188ae2-2173-4137-9236-19a137a402c2,1004,3.0,3.0,3.0,0.423182,0.654178,0.248452,0.308815,0.411340,0.289848,0.523879,0.772754,0.381760,3.381836,4.306885,3.177246,1456.0,1456.0,1999.0,0.391087,8.0,20.0,0.0,-0.419901,-1.362703,-1.080864,0.689618,0.772720,0.416970,1.006023,0.050483,1.008632,3.467728,732.0,298.0,2.564488,66.608393,-0.732321,1.100528,0.050483,-1.561527e-04,0.505173,23.538420,1.4,95.0,32700.652632,17282.320529,2.986786e+08,35057.0,356.0,1092.0,232.0,232.0,8.792525e-04,6.602315e-04,0.002053,1.173877e-03,0.375906,0.282268,0.877773,0.501866
4,4,19a3e9ea-fce1-40b7-9457-2618970beb7b,1004,1.0,1.0,1.0,0.524878,0.509686,0.314112,0.297205,0.423762,0.342080,0.603181,0.662837,0.464419,3.525391,5.834961,5.738526,1315.0,1315.0,1276.0,0.490333,12.0,21.0,3.0,-0.224772,-0.573604,-0.450277,0.042139,-0.592047,-0.327536,1.006283,0.064806,1.009389,5.859008,1149.0,298.0,11.025064,478.484688,-1.725859,2.106261,0.064806,-5.454373e-05,1.288797,121.938505,1.8,47.0,24490.765957,17779.355634,3.161055e+08,17279.0,4270.0,165.0,199.0,199.0,1.846220e-03,1.196197e-03,0.003711,1.864816e-03,0.443346,0.287251,0.891157,0.447811


Subject wise split data 

In [0]:
df_train_1051  = df_train[df_train['subject_id']==1051]
df_test_1051  = df_test[df_test['subject_id']==1051]

df_test_1051.shape

(64, 61)

Standardise train and test features

In [0]:
x_train_features = pd.DataFrame(df_train_1051.iloc[:,6:])
x_test_features = pd.DataFrame(df_test_1051.iloc[:,3:])

# Fill missing values with median
x_train_features = x_train_features.fillna(x_train_features.median())
x_test_features = x_test_features.fillna(x_test_features.median())

# Standardise the featres
x_train_features = StandardScaler().fit_transform(x_train_features)
x_test_features = StandardScaler().fit_transform(x_test_features)

y_train = pd.DataFrame(df_train_1051.loc[:,'on_off'])
print(x_train_features)


[[ 0.20036197 -0.50945473  2.11189393 ... -1.18174774 -1.29518464
  -1.21949609]
 [ 1.37677271 -0.55587012 -0.45097824 ... -0.71412327  1.42978735
  -0.6361611 ]
 [-0.98017246  0.39463349 -0.58670064 ...  0.781045   -0.1426282
   0.21257124]
 ...
 [ 2.0036427  -0.11324335 -1.0113498  ...  0.3531239   0.15242565
   0.34052399]
 [-1.25402065 -0.29070124 -0.80812151 ... -0.38150045  0.94060891
  -0.423207  ]
 [-0.1431886   0.84172797 -1.03133326 ...  0.0296388  -0.03141248
  -0.13380703]]


Apply PCA on features

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)

pct_train = pca.fit_transform(x_train_features)
pct_test = pca.fit_transform(x_test_features)


principal_train_df = pd.DataFrame(pct_train,columns=['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10'])
principal_test_df = pd.DataFrame(pct_test,columns=['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10'])


principal_train_df['on_off'] = y_train['on_off'].values
principal_test_df.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,-1.738597,-0.695189,-0.234482,-1.068599,-2.597167,-1.031773,0.357683,-0.308498,0.859524,1.660927
1,-1.643140,-2.891753,-1.608947,-2.024884,-1.251867,-0.530492,0.345357,-1.177378,-0.279558,0.243824
2,-1.632876,-4.503770,1.486564,-0.065959,1.548117,-0.573316,1.461216,-1.403195,-0.236852,0.072768
3,-2.971797,-2.374092,2.873241,1.916797,-1.801477,3.086075,-1.043113,-1.020546,-1.725453,0.674156
4,1.722105,-0.816841,-0.901281,0.215590,-2.181360,-1.197754,-1.316444,-1.242219,-1.535401,-1.288297


In [0]:
pca.explained_variance_ratio_

array([0.29211557, 0.12860994, 0.09624368, 0.087212  , 0.05307112,
       0.04625019, 0.03691091, 0.03416767, 0.0251232 , 0.02296913])

Data Preparation for Modeling

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

principal_train_df = principal_train_df.fillna(principal_train_df.median())
principal_test_df = principal_test_df.fillna(principal_test_df.median())
feature_cols = ['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10']
x_train = principal_train_df[feature_cols]
y_train = principal_train_df.on_off
x_test = principal_test_df[feature_cols]

## Train Data Modeling



## Test Modeling

Extra Tree Classifier

In [0]:
 # Extra tree classifier
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(x_train, y_train)
y_pred_ensemble = clf.predict(x_test)


pd.DataFrame(y_pred_ensemble)[0].value_counts()


# save to csv
df_test_1051['prediction'] = y_pred_ensemble
df_1051_predict= df_test_1051[['measurement_id','prediction']].copy()
User = 1051
savedirtest = '/content/drive/My Drive/BEAT_PD/Data/CIS-PD/test_data/Processed_files/Predictions/CIS_PD_Omkar/'
filename = str(savedirtest) + 'User_' + str(User) +' ET' + '.csv'
df_1051_predict.to_csv(filename, sep=',')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Multinomial Logistic Regression

In [0]:
# Multinomial Logistic Regression
logreg = LogisticRegression(solver='lbfgs',multi_class='multinomial')
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)

# save to csv
df_test_1051['prediction'] = y_pred
df_1051_predict= df_test_1051[['measurement_id','prediction']].copy()
User = 1051
savedirtest = '/content/drive/My Drive/BEAT_PD/Data/CIS-PD/test_data/Processed_files/Predictions/CIS_PD_Omkar/'
filename = str(savedirtest) + 'User_' + str(User) +' MLR' + '.csv'
df_1051_predict.to_csv(filename, sep=',')

XGBoost

In [0]:
# XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)
y_pred_xgb = xgb_model.predict(x_test)

# save to csv
df_test_1051['prediction'] = y_pred_xgb
df_1051_predict= df_test_1051[['measurement_id','prediction']].copy()
User = 1051
savedirtest = '/content/drive/My Drive/BEAT_PD/Data/CIS-PD/test_data/Processed_files/Predictions/CIS_PD_Omkar/'
filename = str(savedirtest) + 'User_' + str(User) +' XGB' + '.csv'
df_1051_predict.to_csv(filename, sep=',')